In [ ]:
import pandas as pd
import numpy as np
import datatable as dt
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc
import optuna

print (xgb.__version__)

In response to https://www.kaggle.com/c/tabular-playground-series-oct-2021/discussion/280986



# Memory Reduction

* This memory reduction part taken from https://www.kaggle.com/azzamradman/tps-10-single-xgboost/notebook
  amazing notebook. Please upvote it if you like this part.

In [ ]:
def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [ ]:
%%time

### Changed this from DataTable to basic pandas read_csv since I don't want any cols to be of type boolean
train = pd.read_csv('../input/tabular-playground-series-oct-2021/train.csv')
train = reduce_memory_usage(train)
test = pd.read_csv('../input/tabular-playground-series-oct-2021/test.csv')
test = reduce_memory_usage(test)

ss = dt.fread('../input/tabular-playground-series-oct-2021/sample_submission.csv').to_pandas()
ss = reduce_memory_usage(ss)

In [ ]:
## Based on the way this is loaded I'm expecting these to be blank

bool_cols_train = []
for i, col in enumerate(train.columns):
    if train[col].dtypes == bool:
        bool_cols_train.append(i)

In [ ]:
## Just checking that it was empty
print (bool_cols_train)

In [ ]:
## Based on the way this is loaded I'm expecting these to be blank

bool_cols_test = []
for i, col in enumerate(test.columns):
    if train[col].dtypes == bool:
        bool_cols_test.append(i)

In [ ]:
## Meaningless code for this test
# train.iloc[:, bool_cols_train] = train.iloc[:, bool_cols_train].astype(int)
# test.iloc[:, bool_cols_test] = test.iloc[:, bool_cols_test].astype(int)

In [ ]:
print("Train set shape", train.shape, "\n", "Test set shape", test.shape)

In [ ]:
train.head()
### I really just want to looks at f276 through to f284

In [ ]:
## Will need some slight changes here based on pd.read_csv and still having "id"
X = train.drop(columns=["id", "target"]).copy()
y = train["target"].copy()

X_test = test.drop(columns=["id"]).copy()


# X = train.drop('target', axis=1).copy()
# y = train['target'].copy()
# X_test = test.copy()

del train
del test

In [ ]:
### I talk about this in the introduction section of version 4 of https://www.kaggle.com/joecooper/tps-oct-joes-sandpit
### It is relevant here as well. 
## To be more specific I think it is also relevant to the perceived benefits of Target Encoding potentially showing improvments

# For now I leave this section of code in so we can do a like for like comparison of v1 of this notebook

X['std'] = X.std(axis=1)
X['min'] = X.min(axis=1)
X['max'] = X.max(axis=1)

X_test['std'] = X_test.std(axis=1)
X_test['min'] = X_test.min(axis=1)
X_test['max'] = X_test.max(axis=1)

In [ ]:
## My new section of code for this notebook is to add StandardScaler
## Again I point to my commentary in  https://www.kaggle.com/joecooper/tps-oct-joes-sandpit where I run the scaler. I'm thinkning that the logic is similar to min/max/std FE

## Is running a scaler as useful as TargetEncoding?

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = pd.DataFrame (data=scaler.fit_transform(X), columns=X.columns)
X_test = pd.DataFrame (data=scaler.transform(X_test), columns=X_test.columns)


### And now I'm changing my mind again. I think there are a few benefits to running Scalers !

### Some thought will have to go into the inclusion of the std/min/max features being but through a scaler 



In [ ]:
X.head(10)

In [ ]:
params = {
    'max_depth': 6,
    'n_estimators': 9500,
    'subsample': 0.7,
    'colsample_bytree': 0.2,
    'colsample_bylevel': 0.6000000001,
    'min_child_weight': 56.419807345,
    'reg_lambda': 75.5665191,
    'reg_alpha': 0.11766876,
    'gamma': 0.64078232264,
    'booster': 'gbtree',
    'eval_metric': 'auc',
    'tree_method': 'gpu_hist',
    'predictor': 'gpu_predictor',
    'use_label_encoder': False
    }

In [ ]:
%%time
kf = StratifiedKFold(n_splits=6, shuffle=True, random_state=13)

preds = []
scores = []

for fold, (idx_train, idx_valid) in enumerate(kf.split(X, y)):
    X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]
    
#     params['learning_rate']=0.007
    params['learning_rate']=0.0011
    model1 = XGBClassifier(**params)
    
    model1.fit(X_train,y_train,
              eval_set=[(X_train, y_train),(X_valid,y_valid)],
              early_stopping_rounds=200,
              verbose=False)
    
#     params['learning_rate']=0.01
    params['learning_rate']=0.021012
    model2 = XGBClassifier(**params)
    
    model2.fit(X_train,y_train,
              eval_set=[(X_train, y_train),(X_valid,y_valid)],
              early_stopping_rounds=200,
              verbose=False,
              xgb_model=model1)
    
#     params['learning_rate']=0.05
    params['learning_rate']=0.007
    model3 = XGBClassifier(**params)
    
    model3.fit(X_train,y_train,
              eval_set=[(X_train, y_train),(X_valid,y_valid)],
              early_stopping_rounds=200,
              verbose=False,
              xgb_model=model2)
    
    pred_valid = model3.predict_proba(X_valid)[:,1]
    fpr, tpr, _ = roc_curve(y_valid, pred_valid)
    score = auc(fpr, tpr)
    scores.append(score)
    
    print(f"Fold: {fold + 1} Score: {score}")
    print('||'*40)
    
    test_preds = model3.predict_proba(X_test)[:,1]
    preds.append(test_preds)
    
print(f"Overall Validation Score: {np.mean(scores)}")

In [ ]:
predictions = np.mean(np.column_stack(preds),axis=1)

ss['target'] = predictions
ss.to_csv('./xgb.csv', index=False)
ss.head()